In [1]:
import pandas as pd
import numpy as np
import random
import math

from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-train-dataset.csv')
test_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-test-dataset.csv')
val_data = pd.read_csv('/content/drive/My Drive/LOCS/combined-swell-classification-eda-validation-dataset.csv')

train_X = train_data        # 78788
test_X = test_data          # 9848    
val_X = val_data            # 9848

# train_Y, test_Y, val_Y
train_Y = train_X["condition"]
test_Y = test_X["condition"]
val_Y = val_X["condition"]

In [4]:
train_X = train_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])
test_X = test_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])
val_X = val_X.drop(columns=['NasaTLX class', 'Condition Label', 'NasaTLX Label', 'condition', 'subject_id'])

In [5]:
val_X

,MEAN,MAX,MIN,RANGE,KURT,SKEW,MEAN_1ST_GRAD,STD_1ST_GRAD,MEAN_2ND_GRAD,STD_2ND_GRAD,ALSC,INSC,APSC,RMSC,MIN_PEAKS,MAX_PEAKS,STD_PEAKS,MEAN_PEAKS,MIN_ONSET,MAX_ONSET,STD_ONSET,MEAN_ONSET,MEAN_LOG,INSC_LOG,APSC_LOG,RMSC_LOG,RANGE_LOG,ALSC_LOG,MIN_LOG,MEAN_1ST_GRAD_LOG,MEAN_2ND_GRAD_LOG,MIN_LOG_LOG,MEAN_1ST_GRAD_LOG_LOG,MEAN_2ND_GRAD_LOG_LOG,APSC_LOG_LOG,ALSC_LOG_LOG,APSC_BOXCOX,RMSC_BOXCOX,RANGE_BOXCOX,MEAN_YEO_JONSON,SKEW_YEO_JONSON,KURT_YEO_JONSON,APSC_YEO_JONSON,MIN_YEO_JONSON,MAX_YEO_JONSON,MEAN_1ST_GRAD_YEO_JONSON,RMSC_YEO_JONSON,STD_1ST_GRAD_YEO_JONSON,RANGE_SQRT,RMSC_SQUARED,MEAN_2ND_GRAD_CUBE,INSC_APSC
0,0.002115,0.005906,0.000719,0.005187,0.332256,1.100373e+00,-0.000193,0.001091,-0.000047,0.001039,27.000042,0.059207,6.547516e-06,0.002559,1.885254,34.788086,7.661684,10.076063,1.601074,35.041504,7.585563,10.098416,0.002112,0.057520,6.547494e-06,0.002556,0.005174,3.332206,0.000719,-0.000193,-0.000047,0.000719,-0.000193,-0.000047,6.547473e-06,1.466077,-0.197815,-3.064959,-5.711738,0.001567,1.081641e+00,0.306122,5.268609e-06,0.000484,0.004059,-0.000190,0.001945,0.000867,0.072022,6.547516e-06,-1.035915e-13,9042.612495
1,0.000684,0.001702,0.000170,0.001531,1.243869,9.856507e-01,-0.000003,0.000225,-0.000004,0.000190,60.000008,0.041696,5.839452e-07,0.000764,1.146484,20.878906,4.448682,4.829500,0.824219,22.443848,4.335379,4.806063,0.000683,0.040851,5.839450e-07,0.000764,0.001530,4.110874,0.000170,-0.000003,-0.000004,0.000170,-0.000003,-0.000004,5.839449e-07,1.631370,-0.176317,-3.632465,-3.084656,0.000655,7.818591e-01,0.951094,5.821819e-07,0.000166,0.001651,-0.000003,0.000732,0.000219,0.039132,5.839452e-07,-4.306237e-17,71404.751480
2,0.002546,0.005376,0.000552,0.004824,-0.682433,3.796566e-01,-0.000189,0.001258,-0.000325,0.001030,16.000029,0.043281,8.317449e-06,0.002884,3.041504,48.081055,16.985667,17.108948,1.875977,48.561523,17.022365,17.157166,0.002543,0.042371,8.317415e-06,0.002880,0.004812,2.833215,0.000552,-0.000189,-0.000325,0.000551,-0.000189,-0.000325,8.317380e-06,1.343704,-31.198753,-15.599374,-10.547039,0.001597,3.782062e-01,-0.762937,7.094744e-06,0.000405,0.003687,-0.000181,0.001858,0.000845,0.069455,8.317449e-06,-3.419505e-11,5203.662320
3,0.031558,0.054244,0.008873,0.045371,-2.000000,2.902214e-16,-0.045371,0.000000,0.000000,0.000000,1.001029,0.063117,1.510548e-03,0.038866,22.445801,22.445801,0.000000,22.445801,20.444336,20.444336,0.000000,20.444336,0.031071,0.061205,1.509408e-03,0.038130,0.044371,0.693661,0.008834,-0.046432,0.000000,0.008795,-0.047545,0.000000,1.508270e-03,0.526893,-8.338908,-4.169454,-3.556716,0.011575,1.696172e-16,-2.551367,5.555661e-04,0.004086,0.028445,-0.038539,0.014724,-0.000000,0.213004,1.510548e-03,0.000000e+00,41.783932
4,0.000976,0.004470,0.000448,0.004022,11.227764,3.126904e+00,0.000032,0.000526,0.000053,0.000397,24.000013,0.024400,1.671810e-06,0.001293,1.749512,34.126465,9.252800,10.558004,1.928711,37.949219,9.673344,10.470540,0.000976,0.024107,1.671809e-06,0.001292,0.004014,3.218876,0.000448,0.000032,0.000053,0.000448,0.000032,0.000053,1.671807e-06,1.439569,-41.315451,-20.657722,-11.187330,0.000808,3.066106e+00,5.922736,1.618068e-06,0.000348,0.003255,0.000033,0.001052,0.000442,0.063418,1.671810e-06,1.511746e-13,14594.687388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844,0.008115,0.034903,0.003557,0.031346,10.407313,3.028513e+00,0.000103,0.004903,-0.000693,0.004925,30.001300,0.251569,1.057402e-04,0.010283,1.536133,63.640137,13.318369,9.645557,1.415039,65.616211,13.767994,9.632454,0.008082,0.224398,1.057346e-04,0.010230,0.030865,3.434029,0.003551,0.000103,-0.000693,0.003544,0.000103,-0.000694,1.057290e-04,1.489309,-0.176317,-2.898766,-2.260831,0.005096,1.828156e+00,4.076029,6.424191e-05,0.002120,0.020107,0.000103,0.006108,0.002781,0.177047,1.057402e-04,-3.328742e-10,2379.124680
9845,0.009676,0.029187,0.003059,0.026128,1.00

In [6]:
class Node:
    def __init__(self):
        self.split_feature = None
        self.split_point = None
        self.result = None
        self.childs = None
        

In [7]:
class DecisionTree:
    
    def __init__(self):
        self.root = None
        
    def DT_print(self, cur_node = None, cnt = 0):
        if cnt == 0:
            cur_node = self.root
        print(' ' * cnt, "Level ", cnt," :: ", cur_node.split_feature, cur_node.split_point, cur_node.result)
        if cur_node.childs is None:
            return
        for child in cur_node.childs:
            self.DT_print(child, cnt + 1)
        
    def gini_impurity(self, data_Y):  # input label dataset of a group
        impurity = 1
        label_counts = Counter(data_Y)
        for label in label_counts:
            p_of_label = label_counts[label] / len(data_Y)
            impurity -= p_of_label ** 2
        return impurity
    
    def information_gain(self, unsplited_data_Y, splited_data_Y):
        gain = self.gini_impurity(unsplited_data_Y)
        # print(gain)
        # print(splited_data_Y)
        for subset in splited_data_Y:
            # print("-", gini_impurity(subset), " X ( ", len(subset), " / ", len(unsplited_data_Y), " )")
            gain -= self.gini_impurity(subset) * (len(subset)/ len(unsplited_data_Y))
        return gain
    
    def split(self, data_X, data_Y, column):
        data_X_subsets = []    # 분할 후의 data_X 그룹을 저장하는 배열
        data_Y_subsets = []    # 분할 후의 data_Y 그룹을 저장하는 배열
        split_point = 0.0      # 최적 분할 기준 값
        split_point_gain = 0.0  # 최적 분할 지점에서의 information_gain

        data_X = data_X.sort_values(by=column)
        data_Y = data_Y[data_X.index]
        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)

        for i in range(1, len(data_Y)):
            candidate_splited_data_X = []
            candidate_splited_data_Y = []
            if data_Y[i-1] != data_Y[i]:
                # print(i, data_Y[i-1], data_Y[i])
                candidate_point = (data_X[column].iloc[i-1] + data_X[column].iloc[i]) / 2
                candidate_splited_data_Y.append(data_Y[:i])
                candidate_splited_data_Y.append(data_Y[i:])
                gain = self.information_gain(data_Y, candidate_splited_data_Y)
                if gain > split_point_gain:
                    candidate_splited_data_X.append(data_X[:i])
                    candidate_splited_data_X.append(data_X[i:])
                    split_point = candidate_point
                    split_point_gain = gain
                    data_X_subsets = candidate_splited_data_X
                    data_Y_subsets = candidate_splited_data_Y
                    # print("== Updated :: ", split_point_gain, split_point)
        return split_point_gain, split_point, data_X_subsets, data_Y_subsets
    
    def find_best_split(self, data_X, data_Y, bagging=False):
        # print("=-=-New Group=-=-")
        best_feature = ''    # 데이터를 분할 할 feature
        best_gain = 0.0       # 데이터를 특정 feature로 분할했을 때 가장 높게 측정된 information_gain
        best_split_point = 0.0

        columns = data_X.columns
        if bagging:     # feature bagging
          num_feature = math.ceil(math.sqrt(len(data_X.columns)))
          col_indices = np.random.choice(len(data_X.columns), num_feature, replace=False)
          columns = [data_X.columns[col_idx] for col_idx in col_indices]

        for column in columns:
            # print("check column :: ", column)
            gain, split_point = self.split(data_X, data_Y, column)[0:2]
            if gain > best_gain:
                best_gain = gain
                best_feature = column
                best_split_point = split_point
        return best_feature, best_gain, best_split_point
    
    def fit(self, data_X, data_Y, cnt=0, bagging=False):
        root = Node()

        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)

        best_feature, best_gain, best_split_point = self.find_best_split(data_X, data_Y, bagging)
        if best_gain == 0:
            root.result = data_Y[0]
            # print(' ' * cnt, "== No Split ", cnt," :: ", root.result)
            return root
        data_X_subsets, data_Y_subsets = self.split(data_X, data_Y, best_feature)[2:]

        # print(' ' * cnt, "== split ", cnt," :: ", best_feature, best_gain)
        childs = []
        for i in range(len(data_X_subsets)):
            childs.append(self.fit(data_X_subsets[i], data_Y_subsets[i], cnt+1))

        root.split_feature = best_feature
        root.split_point = best_split_point
        root.childs = childs
        if cnt == 0:
            self.root = root
        return root
    
    def predict(self, dataset):
        result = []
        for i in dataset.index:
            cur_node = self.root
            while cur_node.result is None:
                value = dataset.loc[i, cur_node.split_feature]
                if value < cur_node.split_point:
                    cur_node = cur_node.childs[0]
                else :
                    cur_node = cur_node.childs[1]
            result.append(cur_node.result)
        return result
    
    def score(self, data_X, data_Y):
        predict = self.predict(data_X)
        cnt = 0
        for i in range(len(data_Y)):
            if predict[i] == data_Y[i]:
                cnt += 1
                
        return cnt / len(data_Y)

In [8]:
newDT = DecisionTree()
newDT.fit(train_X.head(100), train_Y.head(100))

In [9]:
newDT.score(val_X.head(100), val_Y.head(100))

0.5

In [10]:
class RandomForest:
    def __init__(self):
        self.trees = []
        self.result = None

    def bagging_data(self, data_X, data_Y):
        indices = [random.randint(0, len(data_X)-1) for x in range(len(data_X))]
        data_X_subset = data_X.iloc[indices]
        data_Y_subset = data_Y.iloc[indices]
        data_X_subset = data_X_subset.reset_index(drop=True)
        data_Y_subset = data_Y_subset.reset_index(drop=True)
        return data_X_subset, data_Y_subset

    def fit(self, data_X, data_Y, tree_num = 10):
        self.trees = []
        for i in range(tree_num):
          data_X_subset, data_Y_subset = self.bagging_data(data_X, data_Y)
          tree = DecisionTree()
          tree.fit(data_X_subset, data_Y_subset, bagging=True)
          self.trees.append(tree)

    def predict(self, dataset):
        results = []
        for tree in self.trees:
          result = tree.predict(dataset)
          results.append(result)
        results = np.transpose(results)
        results = [Counter(result).most_common()[0][0] for result in results] # max 값이 두개라면 어떡할지 고민
        return results

    def score(self, data_X, data_Y):
        predict = self.predict(data_X)
        cnt = 0
        for i in range(len(data_Y)):
            if predict[i] == data_Y[i]:
                cnt += 1
                
        return cnt / len(data_Y)

In [11]:
RF = RandomForest()

In [ ]:
RF.fit(train_X, train_Y)

In [ ]:
RF.score(test_X, test_Y)